# LDA, lanent Dirichlet allocation
## 潜在狄利克雷分配

https://github.com/susanli2016/NLP-with-Python/blob/master/LDA_news_headlines.ipynb

In [0]:
import pandas as pd

data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

# dataset: https://www.kaggle.com/therohk/million-headlines/data

In [3]:
len(documents)

1103663

In [4]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


### Data Preprocessing

In [0]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [7]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### Lemmatize example

In [8]:
print(WordNetLemmatizer().lemmatize('went', pos='v'))

go


#### Stemmer Example

In [9]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [0]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [11]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [0]:
processed_docs = documents['headline_text'].map(preprocess)

In [13]:
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

### Bag of words on the dataset

In [0]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [15]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [0]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [17]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(76, 1), (112, 1), (484, 1), (4015, 1)]

In [18]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 484 ("rain") appears 1 time.
Word 4015 ("dampen") appears 1 time.


### TF-IDF

In [0]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [0]:
corpus_tfidf = tfidf[bow_corpus]

In [21]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5893154583024485),
 (1, 0.3892866165028569),
 (2, 0.49651921997736453),
 (3, 0.5046106271280878)]


### Running LDA using Bag of Words

In [0]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [23]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.023*"death" + 0.020*"hospit" + 0.016*"trial" + 0.014*"driver" + 0.012*"street" + 0.011*"flood" + 0.010*"road" + 0.010*"babi" + 0.010*"east" + 0.009*"wall"
Topic: 1 
Words: 0.025*"world" + 0.019*"test" + 0.016*"interview" + 0.014*"australia" + 0.012*"leagu" + 0.010*"close" + 0.010*"darwin" + 0.009*"take" + 0.009*"deal" + 0.009*"star"
Topic: 2 
Words: 0.018*"council" + 0.018*"countri" + 0.017*"nation" + 0.015*"tasmanian" + 0.015*"hour" + 0.014*"labor" + 0.013*"plan" + 0.012*"say" + 0.011*"citi" + 0.011*"state"
Topic: 3 
Words: 0.023*"perth" + 0.020*"women" + 0.018*"miss" + 0.018*"sydney" + 0.013*"protest" + 0.012*"guilti" + 0.011*"search" + 0.011*"water" + 0.011*"john" + 0.010*"continu"
Topic: 4 
Words: 0.028*"elect" + 0.018*"say" + 0.017*"live" + 0.017*"famili" + 0.016*"chang" + 0.011*"park" + 0.011*"polit" + 0.010*"vote" + 0.010*"leader" + 0.010*"christma"
Topic: 5 
Words: 0.036*"trump" + 0.028*"queensland" + 0.017*"australia" + 0.015*"tasmania" + 0.014*"turnbul" + 0

Cool! Can you distinguish different topics using the words in each topic and their corresponding weights?

### Running LDA using TF-IDF

In [0]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [25]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.011*"crash" + 0.011*"kill" + 0.009*"donald" + 0.008*"dead" + 0.007*"attack" + 0.007*"north" + 0.006*"korea" + 0.006*"truck" + 0.006*"wednesday" + 0.006*"monday"
Topic: 1 Word: 0.018*"rural" + 0.008*"grandstand" + 0.008*"sentenc" + 0.008*"plead" + 0.006*"guilti" + 0.006*"nation" + 0.005*"disabl" + 0.005*"track" + 0.005*"drink" + 0.005*"product"
Topic: 2 Word: 0.022*"countri" + 0.020*"hour" + 0.011*"news" + 0.011*"podcast" + 0.007*"abbott" + 0.007*"live" + 0.007*"ash" + 0.006*"mental" + 0.006*"malcolm" + 0.005*"health"
Topic: 3 Word: 0.015*"trump" + 0.009*"govern" + 0.007*"fund" + 0.006*"sport" + 0.006*"council" + 0.006*"plan" + 0.005*"water" + 0.005*"juli" + 0.004*"drought" + 0.004*"land"
Topic: 4 Word: 0.011*"drum" + 0.011*"weather" + 0.007*"david" + 0.007*"climat" + 0.006*"march" + 0.006*"friday" + 0.006*"dairi" + 0.006*"australia" + 0.005*"john" + 0.005*"wild"
Topic: 5 Word: 0.010*"final" + 0.008*"leagu" + 0.008*"world" + 0.006*"australia" + 0.006*"beat" + 0.006*"ope

### Classification of the topics

### Performance evaluation by classifying sample document using LDA Bag of Words model

In [26]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [27]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.3099364936351776	 
Topic: 0.031*"govern" + 0.018*"market" + 0.013*"australian" + 0.013*"share" + 0.013*"break" + 0.012*"warn" + 0.011*"busi" + 0.011*"high" + 0.011*"bank" + 0.009*"fund"

Score: 0.28417813777923584	 
Topic: 0.016*"health" + 0.015*"rural" + 0.013*"power" + 0.012*"servic" + 0.012*"price" + 0.011*"hous" + 0.011*"concern" + 0.011*"communiti" + 0.010*"royal" + 0.009*"farm"

Score: 0.2658781111240387	 
Topic: 0.028*"elect" + 0.018*"say" + 0.017*"live" + 0.017*"famili" + 0.016*"chang" + 0.011*"park" + 0.011*"polit" + 0.010*"vote" + 0.010*"leader" + 0.010*"christma"

Score: 0.020003201439976692	 
Topic: 0.023*"perth" + 0.020*"women" + 0.018*"miss" + 0.018*"sydney" + 0.013*"protest" + 0.012*"guilti" + 0.011*"search" + 0.011*"water" + 0.011*"john" + 0.010*"continu"

Score: 0.02000184915959835	 
Topic: 0.023*"death" + 0.020*"hospit" + 0.016*"trial" + 0.014*"driver" + 0.012*"street" + 0.011*"flood" + 0.010*"road" + 0.010*"babi" + 0.010*"east" + 0.009*"wall"

Score: 0.0200

Our test document has the highest probability to be part of the topic on the top.

### Performance evaluation by classifying sample document using LDA TF-IDF model

In [28]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5878947973251343	 
Topic: 0.015*"trump" + 0.009*"govern" + 0.007*"fund" + 0.006*"sport" + 0.006*"council" + 0.006*"plan" + 0.005*"water" + 0.005*"juli" + 0.004*"drought" + 0.004*"land"

Score: 0.25208669900894165	 
Topic: 0.010*"final" + 0.008*"leagu" + 0.008*"world" + 0.006*"australia" + 0.006*"beat" + 0.006*"open" + 0.006*"tuesday" + 0.005*"decemb" + 0.005*"toni" + 0.005*"star"

Score: 0.020004238933324814	 
Topic: 0.014*"polic" + 0.013*"charg" + 0.013*"murder" + 0.010*"court" + 0.009*"death" + 0.008*"assault" + 0.008*"woman" + 0.008*"alleg" + 0.006*"jail" + 0.006*"arrest"

Score: 0.020002644509077072	 
Topic: 0.014*"interview" + 0.009*"royal" + 0.008*"commiss" + 0.007*"liber" + 0.007*"elect" + 0.007*"michael" + 0.006*"stori" + 0.006*"septemb" + 0.006*"thursday" + 0.006*"senat"

Score: 0.02000254951417446	 
Topic: 0.018*"rural" + 0.008*"grandstand" + 0.008*"sentenc" + 0.008*"plead" + 0.006*"guilti" + 0.006*"nation" + 0.005*"disabl" + 0.005*"track" + 0.005*"drink" + 0.005*"p

Our test document has the highest probability to be part of the topic on the top.

### Testing model on unseen document

In [29]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.3499976098537445	 Topic: 0.028*"elect" + 0.018*"say" + 0.017*"live" + 0.017*"famili" + 0.016*"chang"
Score: 0.18333333730697632	 Topic: 0.031*"govern" + 0.018*"market" + 0.013*"australian" + 0.013*"share" + 0.013*"break"
Score: 0.18333320319652557	 Topic: 0.036*"trump" + 0.028*"queensland" + 0.017*"australia" + 0.015*"tasmania" + 0.014*"turnbul"
Score: 0.1833295226097107	 Topic: 0.025*"world" + 0.019*"test" + 0.016*"interview" + 0.014*"australia" + 0.012*"leagu"
Score: 0.016670528799295425	 Topic: 0.023*"death" + 0.020*"hospit" + 0.016*"trial" + 0.014*"driver" + 0.012*"street"
Score: 0.01666843704879284	 Topic: 0.016*"health" + 0.015*"rural" + 0.013*"power" + 0.012*"servic" + 0.012*"price"
Score: 0.016667399555444717	 Topic: 0.023*"perth" + 0.020*"women" + 0.018*"miss" + 0.018*"sydney" + 0.013*"protest"
Score: 0.01666666753590107	 Topic: 0.018*"council" + 0.018*"countri" + 0.017*"nation" + 0.015*"tasmanian" + 0.015*"hour"
Score: 0.01666666753590107	 Topic: 0.031*"charg" + 0.02

**Reference**:  
https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24
https://medium.com/analytics-vidhya/topic-modeling-using-lda-and-gibbs-sampling-explained-49d49b3d1045
https://medium.com/@lettier/how-does-lda-work-ill-explain-using-emoji-108abf40fa7d